# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

# Terminal commands import files into database
mongoimport --type json -d uk_restaurants -c establishments --drop --jsonArray establishments.json

In [148]:
# Code to drop collection if necessary
# food_db.drop_collection('establishments')

In [149]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint


In [150]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [151]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'uk_restaurants']


In [152]:
# assign the uk_food database to a variable name
food_db = mongo['uk_restaurants']

In [153]:
# review the collections in our new database
print(food_db.list_collection_names())

['establishments']


In [154]:
# Assign collection to a variable
establishments = food_db.establishments

In [155]:
# review a document in the establishments collection
establishments.find_one({})

{'_id': ObjectId('640b815cbcf65e2bced1121c'),
 'FHRSID': 1034540,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-08-17T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode'

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [156]:
# Create a dictionary for the new restaurant data
dining_dict ={
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [157]:
# Insert the new restaurant into the collection
establishments.insert_one(dining_dict)

In [158]:
# Check that the new restaurant was inserted
check = establishments.find({'BusinessName': 'Penang Flavours'})
for food in check:
    print(food)

{'_id': ObjectId('640b818add27ef84aab43cc1'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


In [159]:
# establishments.delete_one({'BusinessName': 'Penang Flavours'})

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [160]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType':'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID':1,'BusinessType':1}
rcc_id = establishments.find_one(query,fields)['BusinessTypeID']
print(rcc_id)
# establishments.find_one('BusinessType':)

1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [161]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({'BusinessName':'Penang Flavours'}, {'$set': {"BusinessTypeID": rcc_id}})

In [162]:
# Confirm that the new restaurant was updated

print(establishments.find_one({'BusinessName':'Penang Flavours'})['BusinessTypeID'])


1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [163]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName':'Dover'})

994

In [164]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName':'Dover'})

In [165]:
# Check if any remaining documents include Dover
establishments.count_documents({'LocalAuthorityName':'Dover'})

0

In [166]:
# Check that other documents remain with 'find_one'
print(establishments.find_one({}))

{'_id': ObjectId('640b815cbcf65e2bced11500'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [173]:
# Change the data type from String to Decimal for longitude
establishments.update_many({},[{'$set':
                                {'geocode.longitude':
                                 {'$toDouble':'$geocode.longitude'}
                                }
                                }
                                ])


In [175]:
# Change the data type from String to Decimal for latitude
establishments.update_many({},[{'$set':
                                {'geocode.latitude':
                                 {'$toDouble':'$geocode.latitude'}
                                }
                                }
                                ])

In [176]:
# Check that the coordinates are now numbers
test = (establishments.find({}))
for i in test:
    print(type(i['geocode']['longitude']),type(i['geocode']['latitude']))

<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class 'float'> <class 'float'>
<class '